In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.parse, re
import matplotlib

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.impute import KNNImputer

In [67]:
df = pd.read_csv("./utils/raw_data.csv", low_memory=False)
df = df.drop(df[df["Type"]=="house group"].index)
df = df.drop(df[df["Type"]=="apartment group"].index)

In [68]:
df.groupby("Type").count()

,id,Available as of,Construction year,Building condition,Street frontage width,Number of frontages,Covered parking spaces,Outdoor parking spaces,Surroundings type,Living area,...,Age of annuitants,EPC description,Venue of the sale,Single session,Terms of visit,Starting price,Percentage rented,Sea view,Floor,Number of floors
Type,,,,,,,,,,,,,,,,,,,,,
apartment,8849,4667,6729,7027,1127,5668,2756,2138,3809,8518,...,8,4,111,111,107,111,0,45,6630,6043
house,9822,5170,6493,8763,4005,8637,5362,4399,3968,9533,...,15,3,73,73,70,73,13,4,0,0


In [69]:
df = df[df["Type"]=="apartment"]

print("Shape: ",df.shape)
for i in df.columns:
    print(i, (40-len(i))*"-", df[i].isna().sum())

Shape:  (8849, 129)
id -------------------------------------- 0
Available as of ------------------------- 4182
Construction year ----------------------- 2120
Building condition ---------------------- 1822
Street frontage width ------------------- 7722
Number of frontages --------------------- 3181
Covered parking spaces ------------------ 6093
Outdoor parking spaces ------------------ 6711
Surroundings type ----------------------- 5040
Living area ----------------------------- 331
Living room surface --------------------- 4476
Dining room ----------------------------- 7853
How many fireplaces? -------------------- 8713
Kitchen type ---------------------------- 1986
Kitchen surface ------------------------- 5656
Bedrooms -------------------------------- 271
Bedroom 1 surface ----------------------- 4189
Bedroom 2 surface ----------------------- 5076
Bedroom 3 surface ----------------------- 7472
Bedroom 4 surface ----------------------- 8657
Bedroom 5 surface ----------------------- 881

In [70]:
for i in df.columns:
    print(i)

id
Available as of
Construction year
Building condition
Street frontage width
Number of frontages
Covered parking spaces
Outdoor parking spaces
Surroundings type
Living area
Living room surface
Dining room
How many fireplaces?
Kitchen type
Kitchen surface
Bedrooms
Bedroom 1 surface
Bedroom 2 surface
Bedroom 3 surface
Bedroom 4 surface
Bedroom 5 surface
Dressing room
Bathrooms
Toilets
Office surface
Office
Professional space
Attic surface
Isolated
Armored door
Surface of the plot
Land is facing street
Wooded land
Plot at rear
Flat land
Width of the lot on the street
Connection to sewer network
Gas, water & electricity
Garden surface
Garden orientation
Caretaker
Elevator
Accessible for disabled people
Intercom
Secure access / alarm
Air conditioning
TV cable
Visio phone
Jacuzzi
Sauna
Swimming pool
Internet
Primary energy consumption
Energy class
Unnamed: 54
Reference number of the EPC report
CO₂ emission
Yearly theoretical total energy consumption
Conformity certification for fuel tanks
H

In [71]:
df = df[['id','Price','Zip','Type','Subtype','location',
       'Surroundings type',
       'Living area',
       'Bedrooms','Kitchen type','Bathrooms',
       'Building condition',
       'Construction year', 
       'Number of frontages',
       'Covered parking spaces', 'Outdoor parking spaces', 
       'Swimming pool',
       'Furnished',
       'How many fireplaces?',
       'Terrace','Terrace surface',
       'Garden','Garden surface',
       'Primary energy consumption','Energy class','Heating type'
       ]]


In [72]:
df = df.rename(columns={
    'location' :'Locality',
    'Transaction Type' : 'Type of sale',
    'Type' :'Type of property',
    'Subtype' : 'Subtype of property',
    'Number of frontages': 'Number of facades',
    'Bedrooms':'Number of rooms',
    'Kitchen type' : 'Fully equipped kitchen',
    'How many fireplaces?' : 'Open fire',
})

In [73]:
df['Locality'] = df['Locality'].apply(urllib.parse.unquote)

def clean_and_convert(column):
    column = column.apply(lambda x: re.sub('\D+', '', str(x)))
    column = column.replace('', np.nan)
    return column

df['Living area'] = clean_and_convert(df['Living area'])
df['Terrace surface'] = clean_and_convert(df['Terrace surface'])
df['Garden surface'] = clean_and_convert(df['Garden surface'])
df['Primary energy consumption'] = clean_and_convert(df['Primary energy consumption'])

In [74]:
# GARDEN AND TERRACE
conditions = [
    (df['Garden']== "Yes"),
    (df["Garden"].isna()) & (df["Garden surface"].isna()),
    (df["Garden surface"].notna())
    ]
values = [1, 0, 1]
df['Garden'] = np.select(conditions, values)

df.loc[(df["Garden"] == 0 ) & (df["Garden surface"].isna()), 'Garden surface'] = 0

conditions = [
    (df['Terrace']== "Yes"),
    (df["Terrace"].isna()) & (df["Terrace surface"].isna()),
    (df["Terrace surface"].notna())
    ]
values = [1, 0, 1]
df['Terrace'] = np.select(conditions, values)

df.loc[(df["Terrace"] == 0 ) & (df["Terrace surface"].isna()), 'Terrace surface'] = 0

In [75]:
def nan_replacement(column):
    column = column.replace("Yes",1)
    column = column.replace("No",0)
    column = column.replace('', np.nan).fillna(0)
    return column

df['Furnished'] = nan_replacement(df['Furnished'])
df['Swimming pool'] = nan_replacement(df['Swimming pool'])
df['Open fire'] = nan_replacement(df['Open fire'])

In [76]:
# Mapping dictionary for replacing values in the "kitchen" column
kitchen_mapping = {
    # np.nan: -1,
    'Not installed': 0,
    'Installed': 1,
    'Semi equipped': 2,
    'Hyper equipped': 3,
    'USA uninstalled' :0,
    'USA installed': 1,
    'USA semi equipped': 2,
    'USA hyper equipped' :3
}
# Replace values in the "Kitchen type" column with corresponding numbers and create a new column called "Kitchen values"
df['Kitchen values'] = df['Fully equipped kitchen'].map(kitchen_mapping).fillna(df['Fully equipped kitchen'])

building_cond_mapping = {
    # np.nan: -1,
    'To restore': 0,
    'To be done up': 2,
    'Just renovated': 3,
    'To renovate': 1,
    'Good': 3,
    'As new' :4
}

df['Building Cond. values'] = df['Building condition'].map(building_cond_mapping).fillna(df['Building condition'])

df = df.loc[:, ~df.columns.isin(['Fully equipped kitchen','Building condition'])]

In [77]:
df.isna().sum()

id                               0
Price                            0
Zip                              0
Type of property                 0
Subtype of property              0
Locality                         0
Surroundings type             5040
Living area                    331
Number of rooms                271
Bathrooms                      991
Construction year             2120
Number of facades             3181
Covered parking spaces        6093
Outdoor parking spaces        6711
Swimming pool                    0
Furnished                        0
Open fire                        0
Terrace                          0
Terrace surface               1568
Garden                           0
Garden surface                 602
Primary energy consumption    1768
Energy class                     0
Heating type                  2216
Kitchen values                1986
Building Cond. values         1822
dtype: int64

In [78]:
df = df.drop(df[df["Living area"].isna()].index)

In [79]:
conditions = [
    (df["Covered parking spaces"].notna()) & (df["Outdoor parking spaces"].notna()),
    (df["Covered parking spaces"].isna()) & (df["Outdoor parking spaces"].isna()),
    (df["Covered parking spaces"].isna()) & (df["Outdoor parking spaces"].notna()),
    (df["Covered parking spaces"].notna()) & (df["Outdoor parking spaces"].isna())
    ]
values = [(df["Covered parking spaces"]+df["Outdoor parking spaces"]), 0, df["Outdoor parking spaces"],df["Covered parking spaces"]]
df['Parking'] = np.select(conditions, values)

df = df.loc[:, ~df.columns.isin(["Covered parking spaces","Outdoor parking spaces"])]

In [80]:
def get_province(zip_code):
    if 1000 <= zip_code <= 1299:
        return 'Brussels Capital Region'
    elif 1300 <= zip_code <= 1499:
        return 'Walloon Brabant'
    elif 1500 <= zip_code <= 1999 or 3000 <= zip_code <= 3499:
        return 'Flemish Brabant'
    elif 2000 <= zip_code <= 2999:
        return 'Antwerp'
    elif 3500 <= zip_code <= 3999:
        return 'Limburg'
    elif 4000 <= zip_code <= 4999:
        return 'Liège'
    elif 5000 <= zip_code <= 5999:
        return 'Namur'
    elif 6000 <= zip_code <= 6599 or 7000 <= zip_code <= 7999:
        return 'Hainaut'
    elif 6600 <= zip_code <= 6999:
        return 'Luxembourg'
    elif 8000 <= zip_code <= 8999:
        return 'West Flanders'
    elif 9000 <= zip_code <= 9999:
        return 'East Flanders'
    else:
        return 'Unknown'
        
df['Province'] = df['Zip'].apply(get_province)

In [81]:
df = df.astype({"Price":"float",
                "Number of rooms":"float",
                "Living area":"float",
                "Terrace surface":"float",
                "Garden surface":"float",
                "Number of facades":"float",
                "Primary energy consumption":"float"})

In [82]:
aptdf = df.copy()

print("Apartment DataFrame shape (before): ",aptdf.shape)
print("Apartment data min (with outliers): ",aptdf['Price'].min())
print("Apartment data max (with outliers): ",aptdf['Price'].max())

# Remove outliers
def remove_outliers(df, columns, n_std) -> pd.DataFrame:
    for col in columns:
        print('Working on column: {}'.format(col))
        mean = df[col].mean()
        sd = df[col].std()
        df = df[(df[col] <= mean+(n_std*sd))]
    return df

aptdf = remove_outliers(aptdf, ['Price'], 4)

print("_"*30)
print("Apartment DataFrame shape (after removing): ",aptdf.shape)
print("Apartment data min (without outliers): ", aptdf['Price'].min())
print("Apartment data max (without outliers): ", aptdf['Price'].max())

Apartment DataFrame shape (before):  (8518, 26)
Apartment data min (with outliers):  10000.0
Apartment data max (with outliers):  8100000.0
Working on column: Price
______________________________
Apartment DataFrame shape (after removing):  (8407, 26)
Apartment data min (without outliers):  10000.0
Apartment data max (without outliers):  2320000.0


In [83]:
print(aptdf.shape)
apt_df = remove_outliers(aptdf, ['Living area'], 3)
print(apt_df.shape)

(8407, 26)
Working on column: Living area
(8373, 26)


In [87]:
apt_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,8373.0,1.041761e+07,2.690999e+05,7735869.0,10383966.0,10505993.0,10563239.0,1.059738e+07
Price,8373.0,3.919283e+05,3.171500e+05,10000.0,219000.0,295000.0,434000.0,2.320000e+06
Zip,8373.0,4.469984e+03,3.312840e+03,1000.0,1180.0,3000.0,8370.0,9.990000e+03
Living area,8373.0,1.040529e+02,4.875070e+01,5.0,75.0,95.0,120.0,4.010000e+02
Number of rooms,8110.0,2.117509e+00,8.221419e-01,1.0,2.0,2.0,3.0,1.400000e+01
Bathrooms,7471.0,1.255789e+00,5.104251e-01,1.0,1.0,1.0,1.0,8.000000e+00
Construction year,6395.0,1.990116e+03,3.353469e+01,1753.0,1969.0,2000.0,2020.0,2.025000e+03
Number of facades,5409.0,2.417822e+00,8.891732e-01,1.0,2.0,2.0,3.0,2.500000e+01
Swimming pool,8373.0,7.285322e-03,8.504769e-02,0.0,0.0,0.0,0.0,1.000000e+00
Furnished,8373.0,6.353756e-02,2.439419e-01,0.0,0.0,0.0,0.0,1.000000e+00


In [88]:
apt_df.groupby("Number of rooms").count()

,id,Price,Zip,Type of property,Subtype of property,Locality,Surroundings type,Living area,Bathrooms,Construction year,...,Terrace surface,Garden,Garden surface,Primary energy consumption,Energy class,Heating type,Kitchen values,Building Cond. values,Parking,Province
Number of rooms,,,,,,,,,,,,,,,,,,,,,
1.0,1699,1699,1699,1699,1699,1699,760,1699,1426,1187,...,1437,1699,1599,1369,1699,1257,1307,1365,1699,1699
2.0,4159,4159,4159,4159,4159,4159,1819,4159,3753,3247,...,3438,4159,3873,3305,4159,3173,3236,3285,4159,4159
3.0,1963,1963,1963,1963,1963,1963,818,1963,1822,1535,...,1602,1963,1812,1550,1963,1461,1577,1532,1963,1963
4.0,228,228,228,228,228,228,78,228,210,174,...,178,228,215,205,228,178,189,199,228,228
5.0,33,33,33,33,33,33,8,33,28,25,...,26,33,32,25,33,27,24,27,33,33
6.0,20,20,20,20,20,20,5,20,16,16,...,20,20,16,18,20,19,15,17,20,20
7.0,2,2,2,2,2,2,1,2,2,2,...,2,2,1,2,2,2,2,1,2,2
8.0,4,4,4,4,4,4,1,4,3,3,...,3,4,4,3,4,4,3,3,4,4
10.0,1,1,1,1,1,1,0,1,0,1,...,1,1,1,0,1,1,0,1,1,1


In [95]:
apt_df = apt_df.drop(apt_df[apt_df["Number of rooms"]>6].index)

In [ ]:
# def one_convert_to_nan(column):
#     column = column.replace(1.0, np.nan)
#     return column

In [96]:
knn_df = apt_df.loc[:, ~apt_df.columns.isin(["Price","Type of property","Subtype of property","Locality","Surroundings type","Energy class","Heating type","Province"])]
other = ['id', 'Zip', 'Living area', 'Number of rooms',
       'Bathrooms', 'Construction year', 'Number of facades', 'Swimming pool',
       'Furnished', 'Open fire', 'Terrace', 'Terrace surface', 'Garden',
       'Garden surface', "Primary energy consumption",'Kitchen values', 'Building Cond. values', 'Parking']

impute_knn = KNNImputer(n_neighbors=5)

knn_df = impute_knn.fit_transform(knn_df).astype(float)

#Creating dfs with missing values filled in 
imputed_houses = pd.DataFrame(knn_df, columns= other)

#Creating dfs with prices
new_apts = apt_df[['id',"Price","Type of property","Subtype of property","Locality","Surroundings type","Energy class","Heating type","Province"]]

#Merging dfs (with prices and without prices (with other values filled in))
complete_apts = pd.merge(new_apts, imputed_houses,on='id')


In [97]:
complete_apts.isna().sum()


id                               0
Price                            0
Type of property                 0
Subtype of property              0
Locality                         0
Surroundings type             4750
Energy class                     0
Heating type                  2056
Province                         0
Zip                              0
Living area                      0
Number of rooms                  0
Bathrooms                        0
Construction year                0
Number of facades                0
Swimming pool                    0
Furnished                        0
Open fire                        0
Terrace                          0
Terrace surface                  0
Garden                           0
Garden surface                   0
Primary energy consumption       0
Kitchen values                   0
Building Cond. values            0
Parking                          0
dtype: int64

In [99]:
df_urbain = pd.read_csv('./utils/Urbain.csv')
postcode_set = set(df_urbain['Postcode'])
complete_apts['Urban_value'] = complete_apts['Zip'].apply(lambda x: 1 if x in postcode_set else 0)

In [101]:
complete_apts.groupby("Subtype of property")["id"].count()

Subtype of property
apartment       6285
duplex           541
flat studio      338
ground floor     456
kot               22
loft              89
penthouse        462
service flat     121
triplex           51
Name: id, dtype: int64

In [108]:
apartment = ["apartment","ground floor","loft","service flat","flat studio","kot"]
big_apt = ["penthouse","triplex","duplex"]

complete_apts["Normal_apt"] = complete_apts["Subtype of property"].apply(lambda x: 1 if x in apartment else 0)
complete_apts["Big_apt"] = complete_apts["Subtype of property"].apply(lambda x: 1 if x in big_apt else 0)

In [ ]:
complete_apts.to_csv("./utils/final_apartment.csv")